# 🚀 **Advanced RVC Inference Pro**

**The Ultimate Voice Conversion Notebook**

This notebook is optimized for speed, stability, and ease of use. 

### ✨ Features:
- ⚡ **Ultra-Fast Setup**: Powered by `uv` and `aria2` for maximum speed.
- 📥 **Model Downloader**: Built-in tool to download voice models from links.
- 💾 **Drive Integration**: Seamlessly load/save models to Google Drive.
- 🌐 **Robust Tunneling**: Auto-reconnecting Gradio and Ngrok integration.
- 🛠️ **Auto-Fix**: Automatically installs missing system dependencies (FFmpeg).

**Repository:** [ArkanDash/Advanced-RVC-Inference](https://github.com/ArkanDash/Advanced-RVC-Inference)


In [ ]:
# @title ## 🛠️ 1. Initialize Environment
# @markdown Installs system dependencies and sets up the `uv` package manager.

import os
import subprocess
import time
from pathlib import Path
from IPython.display import clear_output

def run_cmd(cmd, msg=None):
    if msg:
        print(f"👉 {msg}...")
    subprocess.run(cmd, shell=True, check=True)

start_time = time.time()
print("🚀 Initializing Advanced RVC Environment...")

# 1. Install System Dependencies (FFmpeg is critical for RVC)
print("📦 Installing system dependencies (FFmpeg, Aria2)...")
run_cmd("apt-get update -qq && apt-get install -y -qq ffmpeg aria2")

# 2. Install uv for fast python management
print("⚡ Installing uv package manager...")
run_cmd("pip install uv")

# 3. Clone Repository
REPO_URL = "https://github.com/ArkanDash/Advanced-RVC-Inference.git"
PROJECT_DIR = Path("/content/Advanced-RVC-Inference")

if not PROJECT_DIR.exists():
    print(f"📥 Cloning repository from {REPO_URL}...")
    run_cmd(f"git clone {REPO_URL} {PROJECT_DIR}")
else:
    print("🔄 Repository exists. Updating...")
    run_cmd(f"cd {PROJECT_DIR} && git pull")

# 4. Install Python Dependencies via uv
print("🐍 Installing Python libraries (this may take a moment)...")
os.chdir(PROJECT_DIR)

# Install requirements with uv system-wide to work with Colab
requirements_cmd = (
    "uv pip install --system "
    "torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 "
    "--extra-index-url https://download.pytorch.org/whl/cu118 "
    "-r requirements.txt"
)
run_cmd(requirements_cmd)

# Install pyngrok for better tunneling
run_cmd("uv pip install --system pyngrok")

clear_output()
print(f"✅ Setup completed in {time.time() - start_time:.2f} seconds!")
print(f"📂 Working Directory: {os.getcwd()}")


In [ ]:
# @title ## 💾 2. Mount Drive & Sync Models
# @markdown Syncs your models/weights folder with Google Drive so you don't lose them.

mount_drive = True #@param {type:"boolean"}
drive_folder_name = "RVC_Backups" #@param {type:"string"}

import shutil

if mount_drive:
    from google.colab import drive
    if not os.path.exists("/content/drive"):
        print("🔗 Connecting to Google Drive...")
        drive.mount('/content/drive')

    drive_path = Path(f"/content/drive/MyDrive/{drive_folder_name}")
    drive_path.mkdir(parents=True, exist_ok=True)
    
    # Create subfolders on drive
    (drive_path / "weights").mkdir(exist_ok=True)
    (drive_path / "logs").mkdir(exist_ok=True)
    (drive_path / "models").mkdir(exist_ok=True)

    print(f"📁 Drive storage set to: {drive_path}")
    
    # Symlink local folders to Drive
    # This means files saved locally automatically go to Drive
    print("🔄 Creating symlinks...")
    
    folders_to_sync = ["weights", "logs", "models"]
    
    for folder in folders_to_sync:
        local_target = PROJECT_DIR / folder
        drive_target = drive_path / folder
        
        if local_target.exists() and not local_target.is_symlink():
            shutil.rmtree(local_target)
        elif local_target.is_symlink():
            local_target.unlink()
            
        local_target.symlink_to(drive_target)
        print(f"  ✅ Linked {folder} <--> Drive")

else:
    print("⚠️ Drive mounting skipped. Models will be lost when runtime ends.")


In [ ]:
# @title ## 📥 3. Model Downloader
# @markdown Paste a URL (HuggingFace, Pixeldrain, etc.) to download a voice model directly.

model_url = "" #@param {type:"string"}
model_name = "" #@param {type:"string"}

# @markdown *Leave `model_name` empty to attempt auto-detection.*

import zipfile

if model_url:
    print(f"⬇️ Downloading model...")
    
    # Handle Pixeldrain/HuggingFace specific quirks
    if "pixeldrain.com" in model_url:
        model_url = f"https://pixeldrain.com/api/file/{model_url.split('/')[-1]}"
    if "huggingface.co" in model_url and "/blob/" in model_url:
        model_url = model_url.replace("/blob/", "/resolve/")

    output_name = model_name if model_name else "temp_model"
    download_path = PROJECT_DIR / "weights" / f"{output_name}.zip"
    
    # Use aria2 for speed
    subprocess.run(f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -o '{output_name}.zip' -d '{PROJECT_DIR}/weights' '{model_url}'", shell=True)
    
    # Check if it's a zip and extract
    downloaded_file = PROJECT_DIR / "weights" / f"{output_name}.zip"
    
    # If user provided a direct .pth link, aria2 might have named it .zip based on my command above if name wasn't detected
    # Let's try to detect the file type
    
    if downloaded_file.exists():
        try:
            with zipfile.ZipFile(downloaded_file, 'r') as zip_ref:
                print("📦 Extracting zip file...")
                zip_ref.extractall(PROJECT_DIR / "weights")
            downloaded_file.unlink() # remove zip after extraction
            print(f"✅ Model extracted to weights folder!")
        except zipfile.BadZipFile:
            print("📄 File is not a zip, assuming raw .pth file.")
            if not model_name:
                 # Try to rename based on URL if no name provided
                 pass
            print("✅ Model downloaded!")
    else:
        # Sometimes aria2 keeps original filename
        print("✅ Download complete (Original filename preserved).")
else:
    print("ℹ️ No URL provided. Skipping download.")


In [ ]:
# @title ## 🚀 4. Launch Interface
# @markdown Select your preferred method to access the WebUI.

tunnel_method = "gradio" #@param ["gradio", "ngrok"]
ngrok_token = "" #@param {type:"string"}

import os

os.chdir(PROJECT_DIR)

if tunnel_method == "ngrok":
    if not ngrok_token:
        print("❌ Error: Ngrok selected but no token provided.")
        print("👉 Sign up at https://dashboard.ngrok.com/get-started/your-authtoken")
    else:
        from pyngrok import ngrok, conf
        print("🔌 Authenticating with Ngrok...")
        conf.get_default().auth_token = ngrok_token
        conf.get_default().monitor_thread = False
        try:
            ssh_tunnel = ngrok.connect(7865)
            print(f"🌟 Ngrok Tunnel Active: {ssh_tunnel.public_url}")
        except Exception as e:
            print(f"⚠️ Ngrok Error: {e}")

print("\n🔥 Starting RVC WebUI... Please wait...")
print("📋 If using Gradio, look for the 'Running on public URL' link below.")

cmd = "python app.py --port 7865 --listen"

if tunnel_method == "gradio":
    cmd += " --share"

subprocess.run(cmd, shell=True)
